# Variables and Memory

- memory references
    - what variables really are
        - memory management
            - reference counting
            
            - garbage collection
                - dynamic vs static typing
                
                       -mutability and immutability
                           - shared references
                               - variable equality
                                   - everything is an object

## Variables are Memory References

![](./images/img9.jpg)

- Storing and retriving objects from the heap is taken care of for us by `Python Memory Manager`

![](./images/img10.jpg)

- Note: my_var_1 != 10, my_var_1 infact is = to the memory address 0x1000 in this case, but 0x1000 represents the memory address of the data that we are actually interested in.
- my_var_1 is a reference to an object at the memory location to which this name/alias is referencing to.

![](./images/img11.jpg)

- It's importat to understand that variable sin python are references to objects in memory.
- In python we can find out the memory address of a variable using the `id() function`.
     - This will return a base10 number. We can convert this base10 number to hexadecimal using `hex() function`.
     

In [2]:
a = 10
print(hex(id(a)))

0x7fffe07763b0


In [3]:
greeting = 'hello'
print(greeting)

hello


In [4]:
print(id(greeting))

1912861025872


In [5]:
print(hex(id(greeting)))

0x1bd5f66e650


- Bottom variables are just memory addresses. They are not equal to the values that we think they are equal to.

# Reference Counting

![](./images/img_12.jpg)
![](./images/img_13.jpg)

- Now suppose my_var goes aways, either becomes out of scope or my_var assigned to None. Now the reference count goes to 1.

![](./images/img_14.jpg)

- Now lets say other_var also goes away. Now since no variable is referencing to 0x1000 therefore the reference count drops down to 0.

![](./images/img_15.jpg)
- Now at this point the python memory manager recgonizes that since there is no references left, therefore it throws away the object stored at memory address 0x1000. 
    - Now this space can be reused again to store another object.

- This is called Reference Counting, this is done automatically by the python memory manager.

## Finding the Reference Count

- sys.getrefcount(my_var)
     - When we pass my_var to `getrefcount() fucntion` it is actually creating another reference to that same object in memory.
     - There is a downside to using getrefcount(), it always increases the reference count by 1 because simply the act of passing my_var to the function getrefcount() creates another reference to that same variable since variables are passed by reference in python. 

In [29]:
#Uderstading the above concept

def func_1(my_var_test):
    print('Inside func_1: ', hex(id(my_var_test)))
my_var_test = 10
print('Inside main:', hex(id(my_var_test)))
func_1(my_var_test)
# This proves that variables are passed by reference in python.

Inside main: 0x7fffe07763b0
Inside func_1:  0x7fffe07763b0


In [39]:
import sys
sys.getrefcount(my_var_test)
#134

134

- The reason we are getting 134 as the refcount because it is not the refcount of my_var_test. Rather it is the reference count of the value 10. Variable names don't have references; they are the references.

[stackoverflow related thread](https://stackoverflow.com/questions/61738531/why-does-a-new-python-variable-have-a-reference-count-of-108)

In [40]:
a = hex(id(my_var_test))
sys.getrefcount(a)

2

- Here we are getting refcount as 2. One for `a` and the other one is resulting from passing a as a parameter to `sys.getrefcount()` since parameters are passed by reference in python.

### Another method to find the reference count without having the drawbacks

- ctypes.c_long.from_address(address).value
    - here we are just passing the memory address(id()/ an integer), not a reference - does not affect reference count.

In [882]:
#example
import sys

In [883]:
a = [1, 2, 3]

In [884]:
id(a)

1912865986760

In [885]:
sys.getrefcount(a)

2

In [886]:
import ctypes

In [887]:
#defining a wrapper function
def refcount(address: int):
    return ctypes.c_long.from_address(address).value

In [888]:
refcount(id(a))
# Note id(a) is evaluated first, when we are running id(a) the reference count to the memory address is 2; 1 of 'a' and 1 since 
# we passed a as parameter to id() function. Now id() finishes running and returns the memory address and hence reduces the refcount back to 1.
#Therefore by the time we call refcount() id has finished running, and it has released it's pointer to that memory address.
#Now, the refernce count is indeed back to 1

1

In [889]:
 refcount(1912865986760)

1

In [890]:
b = a

In [891]:
id(b)
# a and b are both pointing to the same location in memory

1912865986760

In [892]:
refcount(id(a))

2

In [893]:
c = a
refcount(id(a))

3

In [894]:
c = 10

In [895]:
refcount(id(a))

2

In [896]:
b = None

In [897]:
id(b)

140736958835936

In [898]:
refcount(id(a))

1

In [62]:
# digressing a little bit also note None is a real object in memory any variable assigned as None will have same memory address
q = None
id(q)

140736958835936

In [319]:
a_id = id(a)

In [320]:
a_id

1912865715144

In [479]:
a = None

In [480]:
id(a)

140736958835936

In [526]:
a_id

1912865715144

In [899]:
#Note
a_id = id(a)
a = None
refcount(a_id)

1

In [299]:
a_id

140736958835936

In [938]:
refcount(a_id)

1

In [939]:
refcount(a_id)

1

In [940]:
refcount(a_id)

1

In [1255]:
refcount(a_id)

1

In [68]:
refcount(a_id)

0

In [69]:
refcount(a_id)

2

- This wierd behaviour is because we are using the c library
- when we set a = None; The python memory manager frees up the memory address; it tosses away the object, now this memory address is available for something else.

- In python we never deal with memory address it's very dangerous to do that. You cannot rely on this.
- The above code is just for illustration purposes, you are never going to use it unless you are trying to debug or understand whats going on.

In [694]:
cat = 253

In [695]:
id(cat)

140736959316496

In [696]:
bat = id(cat)

In [697]:
bat

140736959316496

In [698]:
refcount(bat)

30

In [745]:
cat = None

In [700]:
id(cat)

140736958835936

In [701]:
bat

140736959316496

In [881]:
cat = None
refcount(bat)

29